In [2]:
import os
import csv
import statistics
import numpy

In [14]:
def compare_two(x,y, group1, group2, cat):
    error = 0
    false_positive_group1 = 0
    false_negative_group1 = 0
    ppv_group1 = 0
    false_positive_group2 = 0
    false_negative_group2 = 0
    ppv_group2 = 0
    count = 0 
    with open("propublica_full.csv", "r", encoding='utf-8') as file:
        csvreader = csv.DictReader(file)
        for row in csvreader:
            if row[cat] == group1: 
                count += 1
                if int(row["decile_score"]) > x: 
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_group1 += 1
                    elif row["is_recid"] == "1":
                        ppv_group1 += 1
                elif int(row["decile_score"]) < x: 
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_group1 += 1
            if row[cat] == group2: 
                count += 1
                if int(row["decile_score"]) > y:    
                    if row["is_recid"] == "0":
                        error += 1
                        false_positive_group2 += 1
                    elif row["is_recid"] == "1":
                        ppv_group2 += 1
                elif int(row["decile_score"]) < y: 
                    if row["is_recid"] == "1":
                        error += 1
                        false_negative_group2 += 1
    if false_positive_group1 == 0 and ppv_group1 == 0:
        ppv_group1 += 1
    if false_positive_group2 == 0 and ppv_group2 == 0:
        ppv_group2 += 1
    return [100*error/count, 100*abs((false_positive_group1/(false_positive_group1 + ppv_group1)
                                      - false_positive_group2/(false_positive_group2 + ppv_group2)))]

In [15]:
def errorFuncTwo(x, y, group1, group2, cat): 
 for i in range(11): 
     for j in range(11): 
        returned = compare_two(i, j, group1, group2, cat)
        x.append(returned[0])
        y.append(returned[1])

In [16]:
#Filling in the low-mid arrays
low_mid_error = []
low_mid_fp = []
arr = errorFuncTwo(low_mid_error, low_mid_fp, "Less than 25", "25 - 45", "age_cat")
#Filling  in the mid-hi arrays
mid_hi_error = []
mid_hi_fp = []
arr = errorFuncTwo(mid_hi_error, mid_hi_fp, "25 - 45", "Greater than 45", "age_cat")
#Filling  in the low-hi arrays
low_hi_error = []
low_hi_fp = []
arr = errorFuncTwo(low_hi_error, low_hi_fp, "Less than 25", "Greater than 45", "age_cat")

#Filling in Black White array
bw_error = []
bw_fp = []
arr = errorFuncTwo(bw_error, bw_fp, "African-American", "Caucasian", "race")

lowest_fp_error = []
lowest_fp = []
for i in range(len(low_hi_fp)):
    mini_fp = min(low_mid_fp[i], mid_hi_fp[i], low_hi_fp[i])
    if mini_fp == low_mid_fp[i]:
        lowest_fp_error.append(low_mid_error[i])
        lowest_fp.append(low_mid_fp[i])
    elif mini_fp == mid_hi_fp[i]:
        lowest_fp_error.append(mid_hi_error[i])
        lowest_fp.append(mid_hi_fp[i])
    else:
        lowest_fp_error.append(low_hi_error[i])
        lowest_fp.append(low_hi_fp[i])
        
lowest_error = []
lowest_error_fp = []
for i in range(len(low_hi_fp)):
    mini_err = min(low_mid_error[i], mid_hi_error[i], low_hi_error[i])
    if mini_err == low_mid_error[i]:
        lowest_error.append(low_mid_error[i])
        lowest_error_fp.append(low_mid_fp[i])
    elif mini_err == mid_hi_error[i]:
        lowest_error.append(mid_hi_error[i])
        lowest_error_fp.append(mid_hi_fp[i])
    else:
        lowest_error.append(low_hi_error[i])
        lowest_error_fp.append(low_hi_fp[i])
    
avg_error = []
std = []
for i in range(len(low_hi_fp)):
    avg_error.append(statistics.mean([low_mid_error[i],mid_hi_error[i],low_hi_error[i]]))
    std.append(numpy.std([low_mid_error[i],mid_hi_error[i],low_hi_error[i]]))

In [23]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.layouts import row, gridplot

# output to static HTML file
output_file("line.html")

data = {'low_mid_error': low_mid_error,
        'low_mid_fp': low_mid_fp,
        'mid_hi_error': mid_hi_error,
        'mid_hi_fp': mid_hi_fp,
        'low_hi_error': low_hi_error,
        'low_hi_fp': low_hi_fp,
        'avg_error': avg_error,
        'std': std,
        'bw_error': bw_error,
        'bw_fp': bw_fp}


source = ColumnDataSource(data=data)

p1 = figure(title= "Less than 25 and 25 - 45", plot_width=600, plot_height=600, tooltips=[("Error", "@low_mid_error%"), ("False Positive Rate", "@low_mid_fp%")])

# add a circle renderer with a size, color, and alpha
p1.circle(x='low_mid_error', y='low_mid_fp', size=5, source=source, color="navy", alpha=0.5)

p2 = figure(title= "25 - 45 and Greater than 45", plot_width=600, plot_height=600, tooltips=[("Error", "@mid_hi_error%"), ("False Positive Rate", "@mid_hi_fp%")])

# add a circle renderer with a size, color, and alpha
p2.circle(x='mid_hi_error', y='mid_hi_fp', size=5, source=source, color="navy", alpha=0.5)

p3 = figure(title= "Less than 25 and Greater than 45", plot_width=600, plot_height=600, tooltips=[("Error", "@low_hi_error%"), ("False Positive Rate", "@low_hi_fp%")])

# add a circle renderer with a size, color, and alpha
p3.circle(x='low_hi_error', y='low_hi_fp', size=5, source=source, color="navy", alpha=0.5)

p4 = figure(title= "Average error and Standard Deviation", plot_width=600, plot_height=600, tooltips=[("Average Age Error", "@avg_error%"), ("Standard Deviation", "@std")])

# add a circle renderer with a size, color, and alpha
p4.circle(x='avg_error', y='std', size=5, source=source, color="navy", alpha=0.5)

p5 = figure(title= "Black and White", plot_width=600, plot_height=600, tooltips=[("Error", "@bw_error%"), ("False Positive Rate", "@bw_fp")])

# add a circle renderer with a size, color, and alpha
p5.circle(x='bw_error', y='bw_fp', size=5, source=source, color="navy", alpha=0.5)

p6 = figure(title= "Black and White False Positive and Standard Deviation between Age Categories", plot_width=600, plot_height=600, tooltips=[("Standard Deviation", "@std"), ("False Positive Rate", "@bw_fp%"), ("Black White Error", "@bw_error%"), ("Average Age Error", "@avg_error%")])

# add a circle renderer with a size, color, and alpha
p6.circle(x='std', y='bw_fp', size=5, source=source, color="navy", alpha=0.5)

# show the results
show(gridplot([[p1,p2],[p3,p4],[p5,p6]]))